In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)


df_comp = pd.read_csv("./data_storage/spglobal_data/company.csv")
df_price = pd.read_csv('./data_storage/spglobal_data/company_price.csv')
df = pd.read_csv("./data_storage/portfolio_history.csv")


cash_flow = pd.read_csv('./data_storage/cash_flow.csv').set_index('transaction_date')
ex_rate = pd.read_csv('./data_storage/currency.csv')[['Unnamed: 0', 'KRWUSD']].ffill().rename(columns={'Unnamed: 0': 'date'})
ex_rate['date'] = pd.to_datetime(ex_rate['date'])
cash_flow.index.name = 'date'
cash_flow.index = pd.to_datetime(cash_flow.index)
cash_flow = cash_flow.groupby('date').sum()

map = df_comp[['tradingitemid', 'ticker_symbol']].set_index('tradingitemid').to_dict()['ticker_symbol']
start_date = df['date'].min()
end_date = df['date'].max()


In [2]:


df_price = pd.read_csv('./data_storage/spglobal_data/company_price.csv')

df_price['tickersymbol'] = df_price['tradingitemid'].map(map)
df_price = df_price.rename(columns={'pricingdate':'date'})
df_price= df_price[['date', 'tickersymbol', 'priceclose_raw']]


df_price = df_price.pivot(index='date', columns='tickersymbol', values='priceclose_raw').ffill()
df_price.index = pd.to_datetime(df_price.index)
full_range = pd.date_range(start=df_price.index.min(), end=df_price.index.max(), freq="D")
df_price = df_price.reindex(full_range).ffill()
df_price.index.name = "date"

df_price = pd.melt(df_price.reset_index(), id_vars=['date'], value_vars=df_price.columns, var_name='ticker', value_name='price_close')
df_price


,date,ticker,price_close
0,2019-01-02,A069500,26270.00
1,2019-01-03,A069500,26015.00
2,2019-01-04,A069500,26215.00
3,2019-01-05,A069500,26215.00
4,2019-01-06,A069500,26215.00
...,...,...,...
111820,2025-10-17,UAL,99.14
111821,2025-10-18,UAL,99.14
111822,2025-10-19,UAL,99.14
111823,2025-10-20,UAL,101.29


In [3]:

def expand_portfolio_to_daily(df, end_date=None, freq="D"):
    """
    Expand sparse portfolio snapshots to daily rows by duplicating each snapshot
    until the next snapshot date (exclusive). For the last snapshot, expand up to
    `end_date` (inclusive) if provided.
    """
    out = df.copy()

    # 1) normalize and keep original as snapshot_date
    out['date'] = pd.to_datetime(out['date']).dt.normalize()
    out = out.rename(columns={'date': 'snapshot_date'})

    # 2) compute next snapshot date (exclusive upper bound)
    snaps = pd.Index(sorted(out['snapshot_date'].unique()))
    next_map = {}
    for i, d in enumerate(snaps):
        if i + 1 < len(snaps):
            next_map[d] = snaps[i + 1]  # next snapshot (exclusive)
        else:
            if end_date is not None:
                next_map[d] = pd.to_datetime(end_date).normalize() + pd.Timedelta(days=1)
            else:
                next_map[d] = d + pd.Timedelta(days=1)  # only same day

    out['next_date'] = out['snapshot_date'].map(next_map)

    # 3) build per-row ranges on chosen frequency, inclusive start / exclusive end
    def _make_range(row):
        end_excl = row['next_date'] - pd.Timedelta(days=1)
        rng = pd.date_range(row['snapshot_date'], end_excl, freq=freq)
        # ensure at least one row even if freq skips the day (e.g., B on weekend)
        if len(rng) == 0:
            rng = pd.DatetimeIndex([row['snapshot_date']])
        return rng

    out['date'] = out.apply(_make_range, axis=1)

    # 4) explode to daily rows
    expanded = out.explode('date', ignore_index=True)

    # 5) tidy up — drop helper columns, sort
    expanded.drop(columns=['snapshot_date', 'next_date'], inplace=True)
    if 'ticker' in expanded.columns:
        expanded.sort_values(['date', 'ticker'], inplace=True, ignore_index=True)
    else:
        expanded.sort_values(['date'], inplace=True, ignore_index=True)

    return expanded

In [4]:
df = expand_portfolio_to_daily(df)
df = df[['date', 'ticker', 'type', 'market', 'amount','total_value']]

In [5]:
df_price

,date,ticker,price_close
0,2019-01-02,A069500,26270.00
1,2019-01-03,A069500,26015.00
2,2019-01-04,A069500,26215.00
3,2019-01-05,A069500,26215.00
4,2019-01-06,A069500,26215.00
...,...,...,...
111820,2025-10-17,UAL,99.14
111821,2025-10-18,UAL,99.14
111822,2025-10-19,UAL,99.14
111823,2025-10-20,UAL,101.29


In [6]:
new_df = df.merge(df_price[['date','ticker','price_close']], on=['date', 'ticker'], how='left')

new_df['market_value'] = new_df['price_close'] * new_df['amount']
new_df.loc[new_df['type'] == 'Cash', 'market_value'] = new_df.loc[new_df['type'] == 'Cash', 'total_value']

new_df = new_df.merge(ex_rate, on='date', how='left')
new_df['market_value_won'] = new_df['market_value']
new_df.loc[new_df['market'] == 'US', 'market_value_won'] *= new_df.loc[new_df['market'] == 'US', 'KRWUSD'] 

In [7]:
ndf = new_df.groupby('date').sum()['market_value_won']
ndf

date
2020-03-05      811500.00
2020-03-06      811500.00
2020-03-07      811500.00
2020-03-08      811500.00
2020-03-09      811500.00
                 ...     
2025-09-25   298251535.84
2025-09-26   301686303.96
2025-09-27   301686303.96
2025-09-28   301686303.96
2025-09-29   300609621.95
Name: market_value_won, Length: 2035, dtype: float64

In [8]:
print(new_df.loc[new_df['date'] == '2025-02-17'])
print(new_df.loc[new_df['date'] == '2025-02-17']['market_value_won'].sum())
print(ndf)
c = cash_flow.cumsum()
df = ndf.to_frame().join(cash_flow.cumsum(), how='left').ffill()
df['total_value'] = df['market_value_won'] - df['transaction_amount']

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['total_value'], linewidth=2)
plt.title('Total Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Total Value (KRW)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('portfolio_value.png')
plt.close()

df.to_csv('my_return.csv')



            date ticker   type market  amount  total_value  price_close  \
24217 2025-02-17   AAPL  Stock     US    1.00       117.85       244.60   
24218 2025-02-17   AMZN  Stock     US   22.00      4298.55       228.68   
24219 2025-02-17    APA  Stock     US  200.00      4448.00        23.14   
24220 2025-02-17  BRK.B  Stock     US   22.00      8595.01       479.59   
24221 2025-02-17    CAR  Stock     US  414.00     35295.86        90.75   
24222 2025-02-17    DAL  Stock     US   28.00      1037.38        65.39   
24223 2025-02-17    HTZ  Stock     US 1114.00      4282.26         4.05   
24224 2025-02-17   MSFT  Stock     US    1.00       205.74       408.43   
24225 2025-02-17    OXY  Stock     US   73.00      3466.04        48.06   
24226 2025-02-17   TEAM  Stock     US    4.00       766.60       313.10   
24227 2025-02-17    TMF    ETF     US 1707.00     68020.50        42.09   
24228 2025-02-17    TSM  Stock     US   17.00      1589.00       203.90   
24229 2025-02-17   TTWO  

In [10]:
print(df_price.loc[df_price['date']=='2025-02-17'])

             date   ticker  price_close
2238   2025-02-17  A069500     34615.00
4723   2025-02-17  A261240     14225.00
7208   2025-02-17      AAL        15.97
9693   2025-02-17     AAPL       244.60
12178  2025-02-17     ABNB       161.42
14663  2025-02-17     ADBE       460.16
17148  2025-02-17      ALB        81.21
19633  2025-02-17     AMZN       228.68
22118  2025-02-17      APA        23.14
24603  2025-02-17     ATVI        89.55
27088  2025-02-17       BA       184.42
29573  2025-02-17     BABA       124.73
32058  2025-02-17      BAC        46.96
34543  2025-02-17    BRK.B       479.59
37028  2025-02-17        C        84.61
39513  2025-02-17      CAR        90.75
41998  2025-02-17       CB       264.52
44483  2025-02-17      CCL        26.23
46968  2025-02-17      CMG        57.08
49453  2025-02-17      CVX       155.34
51938  2025-02-17      DAL        65.39
54423  2025-02-17     FICO      1799.19
56908  2025-02-17      GEO        27.26
59393  2025-02-17       GM        48.37
